In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install mediapipe opencv-python pandas scikit-learn

In [ ]:
#preprocessing code
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 🔹 Configurable Parameters
INPUT_CSV = "/kaggle/input/data-v1/data.csv"
OUTPUT_CSV = "gesture_all_balanced_augmented.csv"
AUGMENT_NOISE = 0.01        # How much random noise to add to landmarks
MIN_SAMPLES_PER_CLASS = 30  # Desired minimum per gesture
TIME_STEPS = 5              # For sequence creation

# 🔹 Step 1: Load Full Dataset
df = pd.read_csv(INPUT_CSV)
print(f"📥 Loaded dataset with {len(df)} samples and {df['gesture'].nunique()} classes")

# 🔹 Step 2: Count Samples Per Gesture
class_counts = Counter(df["gesture"])
max_dim = df.shape[1] - 1

# 🔹 Step 3: Augment Underrepresented Gestures
augmented_rows = []

for gesture, count in class_counts.items():
    if count >= MIN_SAMPLES_PER_CLASS:
        continue  # Enough samples

    num_needed = MIN_SAMPLES_PER_CLASS - count
    samples = df[df["gesture"] == gesture]

    for _ in range(num_needed):
        row = samples.sample(n=1).iloc[0]
        landmark = row.iloc[1:].values.astype(np.float32)
        noise = np.random.normal(0, AUGMENT_NOISE, size=landmark.shape)
        augmented = landmark + noise
        augmented_rows.append([gesture] + list(augmented))

# 🔹 Combine Original + Augmented
df_aug = pd.DataFrame(augmented_rows, columns=df.columns)
df_final = pd.concat([df, df_aug], ignore_index=True)
df_final.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Saved full + balanced dataset: {OUTPUT_CSV} ({len(df_final)} samples)")

# 🔹 Step 4: Prepare for Transformer (X_seq, y_seq)
X = df_final.iloc[:, 1:].values
y_raw = df_final["gesture"].values

# Label Encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_raw)

# Sequence Preparation
X_seq = np.array([X[i:i + TIME_STEPS] for i in range(len(X) - TIME_STEPS)])
y_seq = y_encoded[TIME_STEPS:]

# One-Hot Encoding
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_seq = one_hot_encoder.fit_transform(y_seq.reshape(-1, 1))

print(f"✅ Final Tensor Shapes — X: {X_seq.shape}, y: {y_seq.shape}")
print(f"🧾 Total Classes: {len(label_encoder.classes_)} | Labels: {list(label_encoder.classes_)}")


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import joblib  # To save encoders

# 🔹 Load Preprocessed Dataset
df = pd.read_csv("/kaggle/working/gesture_balanced.csv")

# 🔹 Prepare Features and Labels
X = df.iloc[:, 1:].values
y_raw = df["gesture"].values

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_raw)

# Save label encoder for later inference
joblib.dump(label_encoder, "label_encoder.pkl")

# Create time-windowed sequences
time_steps = 5
X_seq = np.array([X[i:i + time_steps] for i in range(len(X) - time_steps)])
y_seq = y_encoded[time_steps:]

# One-hot encode the labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_seq = one_hot_encoder.fit_transform(y_seq.reshape(-1, 1))

# Save one-hot encoder for inference if needed
joblib.dump(one_hot_encoder, "one_hot_encoder.pkl")

# Get number of classes
num_classes = y_seq.shape[1]

# 🔹 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq
)

# 🔹 Transformer Model Definition
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.models import Model

def build_simple_transformer(num_classes, time_steps=5, feature_dim=42):
    """
    A compact Transformer model for small gesture datasets.
    """
    inputs = Input(shape=(time_steps, feature_dim))

    # 🔹 Lightweight Attention
    attn = MultiHeadAttention(num_heads=2, key_dim=16)(inputs, inputs)
    attn = LayerNormalization()(attn)
    attn = Dropout(0.2)(attn)

    # 🔹 Global Pooling → Dense
    x = GlobalAveragePooling1D()(attn)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)

    # 🔹 Output
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# 🔹 Build & Train the Model
model = build_simple_transformer(num_classes)
model.summary()

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32
)

# 🔹 Save Trained Model
model.save("gesture_transformer_model.keras")
print("✅ Model training complete and saved as gesture_transformer_model.keras")


In [ ]:
from collections import Counter
print(Counter(df["gesture"]))


In [ ]:
#removing class imbalance
from collections import Counter

min_samples = 10
counts = Counter(df["gesture"])
valid_classes = [cls for cls, count in counts.items() if count >= min_samples]

df_balanced = df[df["gesture"].isin(valid_classes)].reset_index(drop=True)
df_balanced.to_csv("gesture_balanced.csv", index=False)

print(f"✅ Using {len(valid_classes)} balanced classes.")

In [ ]:
!ls /kaggle/working

In [ ]:
#cloning the repository
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

token = user_secrets.get_secret("PAT")

!git clone https://{token}@github.com/atchudhansg/sign-language-data-prep.git

In [ ]:
from kaggle_secrets import UserSecretsClient
import os

#github authentication 
!git config --global user.name "atchudhansg"
!git config --global user.email "116624804+atchudhansg@users.noreply.github.com"

user_secrets = UserSecretsClient()
kaggle_username = user_secrets.get_secret("KAGGLE_USERNAME")
kaggle_key = user_secrets.get_secret("KAGGLE_KEY")

# ✅ Set environment variables so the Kaggle API can find them
os.environ["KAGGLE_USERNAME"] = kaggle_username
os.environ["KAGGLE_KEY"] = kaggle_key

#download the notebook in working directory
!kaggle kernels pull atchusg/machine-vision-gesture-detection -p /kaggle/working

#check if the notebook is downloaded or not
!ls /kaggle/working

!mkdir -p /kaggle/working/sign-language-data-prep/notebooks/kaggle
!mv /kaggle/working/machine-vision-gesture-detection.ipynb /kaggle/working/sign-language-data-prep/notebooks/kaggle/
%cd /kaggle/working/sign-language-data-prep

#commit the stages
!git add .
!git commit -m "Update Kaggle Notebook"

#push to github
token = user_secrets.get_secret("PAT")
!git push https://{token}@github.com/atchudhansg/sign-language-data-prep.git main
